In [1]:
import sys
sys.path.append("..")

In [2]:
from collections import deque
import re

from resources.utils import get_puzzle_input
from resources.tree import Node

In [3]:
class Area(Node):
    def __init__(self, name, start=(0, 0)):
        self.start = start
        self.end = start
        self.directions = []
        self.points = {start: 0}
        self.num_steps = 0
        self.is_cloned = False

        super().__init__(name)

    @property
    def furthest_point(self):
        return max(self.points, key=self.points.get)
        
    @property
    def num_doors(self):
        parent_doors = self.parent.num_doors if self.parent else 0
        my_doors = self.points[self.furthest_point]
        
        return my_doors + parent_doors
        
    def add_direction(self, direction):
        self.directions.append(direction)

        if direction == 'N':
            self.end = self.end[0], self.end[1] + 2               
        elif direction == 'S':
            self.end = self.end[0], self.end[1] - 2
        elif direction == 'E':
            self.end = self.end[0] + 2, self.end[1]
        else:
            self.end = self.end[0] - 2, self.end[1]

        # May have already reached this point if this is a dead end.
        if self.end not in self.points:
            self.points[self.end] = len(self.directions)
            
    def clone(self, clone_parent=True):
        cloned_area = Area(
            name='{}.CLONE'.format(self.name),
            start=self.start
        )
        
        if clone_parent:
            cloned_area.add_parent(self.parent)

        cloned_area.end = self.end
        cloned_area.directions = list(self.directions)
        cloned_area.points = dict(self.points)
        cloned_area.num_steps = self.num_steps

        self.is_cloned = True
        return cloned_area

    def __repr__(self):
        return '<Area 0x:{} name={}, parent={} dirs={} st={} end={}>'.format(
            id(self),
            self.name,
            self.parent.name if self.parent else None,
            ''.join(self.directions),
            self.start,
            self.end
        )

### Part 1

In [4]:
class DoorMap:
    def __init__(self, regexp):
        # remove the ^$
        self.regexp = regexp[1:-1]

        self.num_areas = 1
        self.root = Area(name=0)

        self._create()

    @property
    def max_doors(self):
        return max(area.num_doors for area in self.root.decendants)
                
    def _create_area(self, parent_stack):
        parent = parent_stack[-1]
        child = Area(name=self.num_areas, start=parent.end)
        self.num_areas += 1
        child.add_parent(parent)
        return child  
        
    def _create(self):
        instructions = deque(self.regexp)

        current_area = None
        parent_stack = [self.root]

        while instructions:
            instruction = instructions.popleft()

            # Directions
            if instruction in ('N', 'E', 'S', 'W'):
                if not current_area:
                    current_area = self._create_area(parent_stack)
  
                current_area.add_direction(instruction)
                expecting_sibling = False
    
            elif instruction == '|':
                current_area = None
                expecting_sibling = True
                
            elif instruction == ')':
                current_area = None

                if expecting_sibling:
                    # We didn't get another sibling so we must have
                    # hit dead ends and need to continue from where
                    # we left off. Copy the current area so we don't
                    # mix the roots of the tree.
                    current_area = parent_stack[-1].clone()

                parent_stack.pop()
                expecting_sibling = False
                
            elif instruction == '(':
                parent_stack.append(current_area)
                current_area = None
                expecting_sibling = False
            else:
                raise ValueError('Unknown instruction: {}'.format(instruction))

        
    

In [5]:
door_map = DoorMap('^WNE$')
door_map.root

<Area 0x:4465621144 name=0, parent=None dirs= st=(0, 0) end=(0, 0)>

In [6]:
for area in door_map.root.decendants:
    print(area, area.num_doors)

<Area 0x:4465621312 name=1, parent=0 dirs=WNE st=(0, 0) end=(0, 2)> 3


In [7]:
door_map = DoorMap('^ENWWW(NEEE|SSE(EE|N))$')

In [8]:
for area in door_map.root.decendants:
    print(area, area.num_doors)

<Area 0x:4465393616 name=1, parent=0 dirs=ENWWW st=(0, 0) end=(-4, 2)> 5
<Area 0x:4465392888 name=2, parent=1 dirs=NEEE st=(-4, 2) end=(2, 4)> 9
<Area 0x:4465393112 name=3, parent=1 dirs=SSE st=(-4, 2) end=(-2, -2)> 8
<Area 0x:4465392440 name=4, parent=3 dirs=EE st=(-2, -2) end=(2, -2)> 10
<Area 0x:4465392608 name=5, parent=3 dirs=N st=(-2, -2) end=(-2, 0)> 9



```
In the first example (^WNE$), this would be the north-east corner 3 doors away.
In the second example (^ENWWW(NEEE|SSE(EE|N))$), this would be the south-east corner 10 doors away.
In the third example (^ENNWSWW(NEWS|)SSSEEN(WNSE|)EE(SWEN|)NNN$), this would be the north-east corner 18 doors away.
```

In [9]:
for regexp, expected in [
    ('^WNE$', 3),
    ('^ENWWW(NEEE|SSE(EE|N))$', 10),
    ('^ENNWSWW(NEWS|)SSSEEN(WNSE|)EE(SWEN|)NNN$', 18),
    ('^ESSWWN(E|NNENN(EESS(WNSE|)SSS|WWWSSSSE(SW|NNNE)))$', 23),
    ('^WSSEESWWWNW(S|NENNEEEENN(ESSSSW(NWSW|SSEN)|WSWWN(E|WWS(E|SS))))$', 31)
]:
    assert DoorMap(regexp).max_doors == expected

In [10]:
puzzle_input = get_puzzle_input('/tmp/day_20.txt')

In [11]:
# Check expected start and end
puzzle_input[0][0], puzzle_input[0][-1]

('^', '$')

In [12]:
puzzle_map = DoorMap(puzzle_input[0])

In [13]:
puzzle_map.max_doors

3669

### Part 2

In [14]:
point_distances = {}
for area in puzzle_map.root.decendants:
    furthest_point = area.furthest_point
    distance = area.num_doors
    furthest_distance = area.points[furthest_point]
    
    for point, rel_dist in area.points.items():
        point_dist = distance + rel_dist - furthest_distance
        if point not in point_distances or point_dist < point_distances[point]:
            point_distances[point] = point_dist            

In [15]:
sum(1 for v in point_distances.values() if v >= 1000)

8369